In [1]:
import pandas as pd
CONSTITUENCIES = 41
# Data from https://wybory.gov.pl/sejmsenat2023/pl/dane_w_arkuszach
df = pd.read_csv('wyniki.csv', sep=";").fillna(0)
parties = pd.concat([df, df.apply(['sum'])]).iloc[:, 25:].set_index([pd.Index(range(1, CONSTITUENCIES + 2))])
parties

,Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów,KOMITET WYBORCZY BEZPARTYJNI SAMORZĄDOWCY,KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE,KOMITET WYBORCZY NOWA LEWICA,KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ,KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ,KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI,KOMITET WYBORCZY POLSKA JEST JEDNA,KOMITET WYBORCZY WYBORCÓW RUCHU DOBROBYTU I POKOJU,KOMITET WYBORCZY NORMALNY KRAJ,KOMITET WYBORCZY ANTYPARTIA,KOMITET WYBORCZY RUCH NAPRAWY POLSKI,KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA
1,501870,16748,53958,47715,174643,31770,169540,7496.0,0.0,0.0,0.0,0.0,0.0
2,323360,5808,39215,25806,107797,19478,120188,5068.0,0.0,0.0,0.0,0.0,0.0
3,776054,22412,106624,88089,206899,54132,286713,11185.0,0.0,0.0,0.0,0.0,0.0
4,533919,8905,80426,52959,162603,34266,186914,7846.0,0.0,0.0,0.0,0.0,0.0
5,537597,7758,84308,60473,183131,34232,158719,6710.0,2266.0,0.0,0.0,0.0,0.0
6,648347,10344,102894,37083,294847,54325,131712,14892.0,2250.0,0.0,0.0,0.0,0.0
7,456872,9522,59577,25691,231882,35594,79501,12930.0,2175.0,0.0,0.0,0.0,0.0
8,517041,11954,77933,47911,143530,33672,195091,5794.0,0.0,0.0,1156.0,0.0,0.0
9,456552,5624,54283,55770,122433,25428,187527,5487.0,0.0,0.0,0.0,0.0,0.0
10,396819,8597,54479,25340,184929,30247,86083,5457.0,1687.0,0.0,0.0,0.0,0.0


In [2]:
constituences = pd.read_csv('okregi.csv', sep=";").iloc[:, [0,1,5,6]].set_index('Numer okręgu')
# Idea?: Dostawic Mandaty po reformie - https://demagog.org.pl/wypowiedzi/wybory-do-sejmu-2023-pkw-apeluje-o-zmiane-rozkladu-mandatow/


# Joining results with constituences information
ed = constituences.join(parties) # election data
ed = pd.concat([ed, ed.apply(['sum'])]) #tofix string concat 

# Checking if given party can participate in seats allocation
def pass_threshold(committy):
    supp_share = 100 * ed.loc['sum'][committy] / ed.loc['sum']['Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów']
    threshold = 5 # Regular Committy
    if 'KOALICYJNY' in committy: 
        threshold = 8 # Coalition Committy
    if 'MNIEJSZOŚĆ' in committy:
        threshold = 0 # Minority Commity
    #print(committy, round(supp_share,4), threshold <= supp_share)
    return threshold <= supp_share

# Calculating which comitties pass the threshold
comitties = [ele for ele in list(ed.columns) if 'KOMITET' in ele]
comitties = [ele for ele in comitties if pass_threshold(ele)]

SEATS = ed.loc['sum']['Liczba mandatów']
VOTES = ed.loc['sum']['Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów']
ATTENDANCE = VOTES / ed.loc['sum']['Wyborcy']
ed['True proportion'] = ed['Liczba głosów ważnych oddanych łącznie na wszystkie listy kandydatów'] * SEATS / VOTES
ed['Voter Strength'] = 100*ed['Liczba mandatów'] / ed['True proportion'] - 100


In [3]:
print(SEATS, VOTES, ATTENDANCE, CONSTITUENCIES,  comitties, '\n')

worst = ed.sort_values('Voter Strength').iloc[0]
best = ed.sort_values('Voter Strength').iloc[-1]
diff = float((best['Voter Strength'] + 100) / ((worst['Voter Strength'] + 100)))
print(f"Voters in {best['Siedziba OKW']} have vote {round(diff, 4)}x as strong as voters in {worst['Siedziba OKW']}")


460 21596674 0.7423675016252962 41 ['KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE', 'KOMITET WYBORCZY NOWA LEWICA', 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ', 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ', 'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI', 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA'] 

Voters in Elbląg have vote 2.291x as strong as voters in Warszawa


In [4]:
# Comparison of vote strength
ed.sort_values('Voter Strength').iloc[:, [2,0,16,17]]

,Siedziba OKW,Liczba mandatów,True proportion,Voter Strength
19,Warszawa,20,36.522788,-45.239667
20,Warszawa,12,15.564537,-22.901660
39,Poznań,10,12.695357,-21.231043
3,Wrocław,14,16.529621,-15.303564
13,Kraków,14,16.134897,-13.231553
25,Gdańsk,12,13.126652,-8.582934
27,Bielsko-Biała,9,9.485681,-5.120149
26,Słupsk,14,14.545459,-3.750031
sum,LegnicaWałbrzychWrocławBydgoszczToruńLublinChe...,460,460.000000,0.000000
37,Konin,9,8.929698,0.787281


In [33]:
# Python Dhont (and Sainte-Laguë) allocation algorithm on custom PriorityQueue implementation 

from queue import PriorityQueue

q = PriorityQueue()

class Node:
    def __init__(self, name, votes, seats = 0, runSL = False):
        self.name = name
        self.votes = votes
        self.seats = seats
        self.runSL = runSL # Sainte-Laguë 
        
    def __gt__(self, other):
        if self.runSL: #  Sainte-Laguë 
            return self.votes / (2 * self.seats + 1) < other.votes / (2 * other.seats + 1)
        else:
            return self.votes / (self.seats + 1) < other.votes / (other.seats + 1)
    
    def __eq__(self, other):
        return self.name == other.name
    
    def __str__(self):
        return f"Party {self.name} - {self.votes} votes, holds {self.seats} seats"
    
    def __repr__(self):
        return self.__str__()
    
    def data(self):
        return (self.votes, self.seats, self.votes/(self.seats))

node1 = Node('Aaaa', 1000)
node2 = Node('Bleeeb', 500)
node3 = Node('Cimcirimcim', 100)


q.put(node1)
q.put(node2)
q.put(node3)

k = 25
for _ in range(k):
    next = q.get() 
    next.seats += 1
    q.put(next)

q.queue 


[Party Aaaa - 1000 votes, holds 16 seats,
 Party Cimcirimcim - 100 votes, holds 1 seats,
 Party Bleeeb - 500 votes, holds 8 seats]

In [32]:
from math import floor

def run_dhondt(data, seats, _runSL = False):
    nodes = [Node(name, votes, runSL=_runSL) for (name, votes) in data]
    q = PriorityQueue()
    for node in nodes:
        q.put(node)
    
    for _ in range(seats):
        next = q.get()
        second_last = next
        next.seats += 1
        q.put(next)
        
    last = q.get()
    q.put(last)
    if last.name == second_last.name:
        last = q.get()
        q.put(last)
        
    last = Node(last.name, last.votes, last.seats)
    last.seats += 1

    return ([(node.name, int(node.seats)) for node in q.queue], second_last, last)


def last_seat(win, los):
    return floor(win.votes - los.votes * win.seats / los.seats)

def parse_last_win(win, los):
    return f"{win.name} won last seat over {los.name} by {last_seat(win, los)} votes"
    
def run_election(id, show=True, runSL=False):
    seats = int(ed.loc[id]['Liczba mandatów'])
    data = dict(ed.loc[id])
    data = [(ele,data[ele]) for ele in comitties if data[ele] > 0]

    allocation,win,los = run_dhondt(data, seats, runSL)
    
    if show:
        for key,val in (list(allocation)):
            print(key, "->", val)
        if (not runSL):
            print(parse_last_win(win, los), "\n")
    return allocation

# Pass which constituency
#dict(run_election(19, False))
run_election(7, True)
run_election(7, True, True)
print("\n##### OPOLE ######\n")
run_election(21, True)
run_election(21, True, True)

KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ -> 6
KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE -> 1
KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI -> 2
KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ -> 1
KOMITET WYBORCZY NOWA LEWICA -> 0
KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ won last seat over KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ by 2468 votes 

KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ -> 5
KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE -> 1
KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ -> 1
KOMITET WYBORCZY NOWA LEWICA -> 1
KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI -> 2

##### OPOLE ######

KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI -> 4
KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE -> 1
KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPOD

[('KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI', 3),
 ('KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE',
  1),
 ('KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ', 3),
 ('KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ', 1),
 ('KOMITET WYBORCZY NOWA LEWICA', 1),
 ('KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA', 1)]

In [7]:
import pprint

sum_parties = dict([(name, 0) for name in comitties])

for id in range(1, CONSTITUENCIES + 1):
    result = run_election(id, False)
    for (key, val) in result:
        sum_parties[key] += val
        
print("\nConstituency-independent Dhont'd with pre-allocated seats (current system in Poland)")  
pprint.pprint(sum_parties)


Constituency-independent Dhont'd with pre-allocated seats (current system in Poland)
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 157,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 65,
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 18,
 'KOMITET WYBORCZY NOWA LEWICA': 26,
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 194,
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA': 0}


In [29]:
sum_parties = dict([(name, 0) for name in comitties])

for id in range(1, CONSTITUENCIES + 1):
    result = run_election(id, False, True)
    for (key, val) in result:
        sum_parties[key] += val
        
print("\nConstituency-independent Sainte-Laguë with pre-allocated seats (alternative to system in Poland)")  
pprint.pprint(sum_parties)


Constituency-independent Sainte-Laguë with pre-allocated seats (alternative to system in Poland)
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 149,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 68,
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 38,
 'KOMITET WYBORCZY NOWA LEWICA': 43,
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 161,
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA': 1}


In [34]:
# True support
sum_parties = dict([(name, 0) for name in comitties])
sum_real_votes = 0
for com in comitties:
    sum_parties[com] = round(100 * ed.loc["sum"][com] / VOTES, 4)
    sum_real_votes += ed.loc["sum"][com]
print("Direct support")  
pprint.pprint(sum_parties)

for com in comitties:
    sum_parties[com] = round(100 * ed.loc["sum"][com] / sum_real_votes, 4)
print("\nEffective support")  
pprint.pprint(sum_parties)


print("\nFair seats share")
for com in comitties:
    sum_parties[com] = round(ed.loc["sum"][com] / sum_real_votes * SEATS, 2)
pprint.pprint(sum_parties)


# Global Dhont'd
data_global = [(com, ed.loc["sum"][com]) for com in comitties]
allocation,win,los = run_dhondt(data_global, SEATS)
print("\nGlobal Dhont'd")
pprint.pprint(dict(allocation))

# Global Sainte-Laguë
data_global = [(com, ed.loc["sum"][com]) for com in comitties]
allocation,win,los = run_dhondt(data_global, SEATS, True)
print("\nGlobal Sainte-Laguë")
pprint.pprint(dict(allocation))

Direct support
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 30.6964,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 14.4035,
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 7.1648,
 'KOMITET WYBORCZY NOWA LEWICA': 8.6079,
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 35.3798,
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA': 0.1194}

Effective support
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 31.8521,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 14.9457,
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 7.4346,
 'KOMITET WYBORCZY NOWA LEWICA': 8.932,
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 36.7118,
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA': 0.1239}

Fair seats share
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 146.52,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLS

In [28]:
# Hare-Nemayer rounding
def find_HMpar(ed):
    low = -1
    high = 1
    while True:
        p = (low + high) / 2 
        val = ed['True proportion'].apply(lambda x: round(x + p)).sum()
        
        if val == SEATS * 2:
            return p
        elif val < SEATS * 2:
            low = p
        else:
            high = p
            
paramHM = find_HMpar(ed)
# Updating seats
ed['Liczba mandatów'] = ed['True proportion'].apply(lambda x: round(x + paramHM))

In [10]:
# Updated voter strength

ed['Voter Strength'] = 100*ed['Liczba mandatów'] / ed['True proportion'] - 100
worst = ed.sort_values('Voter Strength').iloc[0]
best = ed.sort_values('Voter Strength').iloc[-1]
diff = float((best['Voter Strength'] + 100) / ((worst['Voter Strength'] + 100)))
print(f"Voters in {best['Siedziba OKW']} have vote {round(diff, 4)}x as strong as voters in {worst['Siedziba OKW']}")

# Comparison of vote strength
ed.sort_values('Voter Strength').iloc[:, [2,0,16,17]]

Voters in Tarnów have vote 1.1127x as strong as voters in Elbląg


,Siedziba OKW,Liczba mandatów,True proportion,Voter Strength
34,Elbląg,6,6.376667,-5.906958
10,Piotrków Trybunalski,8,8.452076,-5.348703
27,Bielsko-Biała,9,9.485681,-5.120149
16,Płock,9,9.426489,-4.524372
22,Krosno,9,9.414327,-4.401030
17,Radom,8,8.332437,-3.989670
35,Olsztyn,8,8.329370,-3.954316
5,Toruń,11,11.450588,-3.935066
4,Bydgoszcz,11,11.372248,-3.273305
11,Sieradz,11,11.355400,-3.129792


In [11]:
# Elections in fairly allocated system with indepentent Dhont'd constituences

sum_parties = dict([(name, 0) for name in comitties])

for id in range(1, CONSTITUENCIES + 1):
    result = run_election(id, False)
    for (key, val) in result:
        sum_parties[key] += val
        
print("\nConstituency-independent Dhont'd with post-allocated seats (my first suggestion)")  
pprint.pprint(sum_parties)


Constituency-independent Dhont'd with post-allocated seats (my first suggestion)
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 159,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 66,
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 15,
 'KOMITET WYBORCZY NOWA LEWICA': 29,
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 191,
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA': 0}


In [12]:
# Squared global allocation

data_global = [(com, ed.loc["sum"][com]) for com in comitties]
sum_votes = sum(val for (_,val) in data_global)
data_global_sq = [(com, val * (1 + val/sum_votes)) for (com, val) in data_global]
allocation,win,los = run_dhondt(data_global_sq, SEATS)
print("\nGlobal Square Dhont'd (my second suggestion)")
pprint.pprint(dict(allocation))


Global Square Dhont'd (my second suggestion)
{'KOALICYJNY KOMITET WYBORCZY KOALICJA OBYWATELSKA PO .N IPL ZIELONI': 152,
 'KOALICYJNY KOMITET WYBORCZY TRZECIA DROGA POLSKA 2050 SZYMONA HOŁOWNI - POLSKIE STRONNICTWO LUDOWE': 62,
 'KOMITET WYBORCZY KONFEDERACJA WOLNOŚĆ I NIEPODLEGŁOŚĆ': 29,
 'KOMITET WYBORCZY NOWA LEWICA': 35,
 'KOMITET WYBORCZY PRAWO I SPRAWIEDLIWOŚĆ': 182,
 'KOMITET WYBORCZY WYBORCÓW MNIEJSZOŚĆ NIEMIECKA': 0}
